In [ ]:
# 훨씬 효율적이다. 지속적으로 api를 요청하지 않음

In [8]:
#여러명의 artist ID를 묶어서 가져오자.

import pickle

file = open("/Users/kangyeolyoun/Desktop/pickle file/aws_mysql_db", "rb")
data = pickle.load(file)

def get_headers(client_id, client_secret):

    endpoint = "https://accounts.spotify.com/api/token"
    encoded = base64.b64encode("{}:{}".format(client_id, client_secret).encode('utf-8')).decode('ascii')

    headers = {
        "Authorization": "Basic {}".format(encoded)
    }

    payload = {
        "grant_type": "client_credentials"
    }

    r = requests.post(endpoint, data=payload, headers=headers)

    access_token = json.loads(r.text)['access_token']

    headers = {
        "Authorization": "Bearer {}".format(access_token)
    }

    return headers

# insert into duplicate 간단화
def insert_row(cursor, data, table):
    placeholders = ", ".join(["%s"] * len(data))
    columns = ", ".join(data.keys())
    key_placeholders = ", ".join(["{0}=%s".format(k) for k in data.keys()]) #id = %s, name=%s 이런식
    sql = "INSERT INTO %s (%s) VALUES (%s) ON DUPLICATE KEY UPDATE %s" % (table, columns, placeholders,\
                                                                         key_placeholders)
    cursor.execute(sql, list(data.values())*2)
    
    
# 원하는 데이터 뽑아오자
import sys
import requests
import base64
import json
import logging
import pymysql
import csv

host = data[0]
port = int(data[1])
username = "kangyeol"
database = data[2]
password = data[3]
client_id = data[4]
client_secret = data[5]


def main():
    
    
    
    try:
        conn = pymysql.connect(host, user=username, passwd=password, db=database,\
                             port=port, use_unicode=True, charset="utf8")
        cursor = conn.cursor()
    except:
        logging.error("could not connect to RDS")
        sys.exit(1)
        
    headers = get_headers(client_id, client_secret)
    
    ## Spotify Search API
#     artists = []
#     with open("../artist_list 복사본.csv")  as f:
#         raw = csv.reader(f)
#         for row in raw:
#             artists.append(row[0])
#     print(artists)

#     for a in artists:
#         params = {
#             "q": a,
#             "type": "artist",
#             "limit": "1"        
#         }
    
#         r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
#         raw = json.loads(r.text)
        
#         artist = {}
        
#         #에러 방지해주기 위해
#         try:
#             if raw["artists"]["items"][0]["name"] == params["q"]:
                
#                 artist.update(
#                     {
#                         "id": raw["artists"]["items"][0]["id"],
#                         "name":raw["artists"]["items"][0]["name"],
#                         "followers":raw["artists"]["items"][0]["followers"]["total"],
#                         "popularity":raw["artists"]["items"][0]["popularity"],
#                         "url": raw["artists"]["items"][0]["external_urls"]["spotify"],
#                         "image_url": raw["artists"]["items"][0]["images"][0]["url"],
#                     }
#                 )
#                 insert_row(cursor, artist, "artists")
                
#         except:
#             logging.error("NO ITEMS FROM SEARCH API")
#             # 에러가 나도 지속적으로 해라
#             continue
        

        
#     conn.commit()


    cursor.execute("SELECT id FROM artists")
    artists = []
    for (id, ) in cursor.fetchall():
        artists.append(id)

    #artist batch를 만들어주자. 50개씩 끊어서 요청
    artist_batch = [artists[i:i+50] for i in range(0, len(artists), 50)]
# 확인해보자
#     print(artist_batch)


    artist_genres = []

    for i in artist_batch:
        ids = ",".join(i)
        URL = "https://api.spotify.com/v1/artists/?ids={}".format(ids)
        
        r = requests.get(URL, headers=headers)
        raw = json.loads(r.text)
#         print(raw)
#         print(len(raw["artists"]))
#         sys.exit(0)
        
        for artist in raw["artists"]:
            for genre in artist["genres"]:
                artist_genres.append(
                {
                    "artist_id": artist["id"],
                    "genre":genre
                }
                )

    for data in artist_genres:      
        insert_row(cursor, data, "artist_genres")
        
    conn.commit()
    sys.exit(0)

    
    r = requests.get("https://api.spotify.com/v1/search", params=params, headers=headers)
    
    # json을 통해서 스트링 형태로 들어온다.
    raw = json.loads(r.text)

    artist = {}
    artist_raw = raw["artists"]["items"][0]
    
    if artist_raw["name"] == params["q"]:
        artist.update({
            "id": artist_raw["id"],
            "name": artist_raw["name"],
            "followers": artist_raw["followers"]["total"],
            "popularity": artist_raw["popularity"],
            "url": artist_raw["external_urls"]["spotify"],
            "image_url": artist_raw["images"][0]["url"]
        })
    
    insert_row(cursor, artist, "artists")
    conn.commit()
    sys.exit(0)
    
     


In [9]:
main()

SystemExit: 0